In [1]:
import numpy as np
import tensorflow as tf
from itertools import product
import random

In [5]:
def decompose_tensor_jennrich(rank3tensor):
    # initialize two random variables
    a = np.random.uniform(size=x.shape[2])
    b = np.random.uniform(size=x.shape[2])
    
    # T_a and T_b
    Ta = sum(x[:, :, i]*a[i] for i in range(x.shape[2]))
    Tb = sum(x[:, :, i]*b[i] for i in range(x.shape[2]))
    
    # eigenvalues of various auxilliary matrices
    eigvals_u, eigvecs_u = np.linalg.eig(np.matmul(Ta, np.linalg.pinv(Tb)))
    eigvals_v, eigvecs_v = np.linalg.eig(np.transpose(np.matmul(np.linalg.pinv(Ta), Tb)))
    
    # pair up reciprocal eigenvalues
    # pair up eigenvalues of Ta and Tb
    idx_pairs = []
    tol = 1e-5

    for i, eigval_u in enumerate(eigvals_u):
        for j, eigval_v in enumerate(eigvals_v):
            if abs(eigval_u - 1/eigval_v) < tol:
                idx_pairs += [(i, j)]
                break

    # solving for third eigenvectors
    nbcomp = len(idx_pairs)
    A = np.zeros((nbcomp*x.shape[2], nbcomp*x.shape[2]))
    B = np.zeros(nbcomp*x.shape[2])
    eqidx = 0
    for k in range(x.shape[2]):
        for i, j in random.choices(list(product(range(x.shape[0]), range(x.shape[1]))), k=nbcomp):
            B[eqidx] = x[i, j, k]
            for ck in range(nbcomp):
                A[eqidx, ck*x.shape[2]+k] = eigvecs_u[i, idx_pairs[ck][0]]*eigvecs_v[j, idx_pairs[ck][1]]
            eqidx += 1
            
    sol = np.linalg.solve(A, B)
    eigvecs_w = sol.reshape((nbcomp, x.shape[2]), order='F')
    
    # rearranging eigenvectors
    rearranged_eigvecs_u = np.zeros(shape=eigvecs_u.shape)
    rearranged_eigvecs_v = np.zeros(shape=eigvecs_v.shape)
    for i, (u_idx, v_idx) in enumerate(idx_pairs):
        rearranged_eigvecs_u[:, i] = eigvecs_u[:, u_idx]
        rearranged_eigvecs_v[:, i] = eigvecs_v[:, v_idx]
    rearranged_eigvecs_w = eigvecs_w
    
    # return values
    return rearranged_eigvecs_u, rearranged_eigvecs_v, rearranged_eigvecs_w

In [6]:
# testing
x1 = np.array([[-1.3, 1], [2, -1.4]], dtype=np.float32)
x2 = np.array([[1, 0], [-1.5, 0.6]], dtype=np.float32)
x = np.zeros((2, 2, 2))
x[:, :, 0] = x1
x[:, :, 1] = x2

In [7]:
eigvecs_u, eigvecs_v, eigvecs_w = decompose_tensor_jennrich(x)

In [8]:
# verfication
xh = np.zeros(x.shape)
for i, j, k in product(range(x.shape[0]), range(x.shape[1]), range(x.shape[2])):
    xh[i, j, k] = sum([eigvecs_u[i, alpha]*eigvecs_v[j, alpha]*eigvecs_w[k, alpha] for alpha in range(eigvecs_u.shape[1])])

In [9]:
xh

array([[[-1.29999995e+00,  1.00000000e+00],
        [ 1.00000000e+00, -3.33066907e-16]],

       [[ 2.00000000e+00, -1.50000000e+00],
        [-1.39999998e+00,  6.00000024e-01]]])

In [10]:
x

array([[[-1.29999995,  1.        ],
        [ 1.        ,  0.        ]],

       [[ 2.        , -1.5       ],
        [-1.39999998,  0.60000002]]])